In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv('bmi.csv')
df.head()

,height,weight,label
0,142,62,fat
1,142,73,fat
2,177,61,normal
3,187,48,thin
4,153,60,fat


In [2]:
df["height"] = df["height"] / 200
df["weight"] = df["weight"] / 100

In [3]:
df = pd.get_dummies(df)

In [4]:
df.head()

,height,weight,label_fat,label_normal,label_thin
0,0.710,0.62,1,0,0
1,0.710,0.73,1,0,0
2,0.885,0.61,0,1,0
3,0.935,0.48,0,0,1
4,0.765,0.60,1,0,0


In [5]:
x_data = df[['height', 'weight']]
y_data = df[['label_thin', 'label_normal', 'label_fat']]

In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [7]:
W1 = tf.Variable(tf.random_normal([2, 10], -1., 1.))
W2 = tf.Variable(tf.random_normal([10, 3], -1., 1.))
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
L1 = tf.add(tf.matmul(X,W1), b1)

In [9]:
L1 = tf.nn.relu(L1)

In [10]:
model = tf.add(tf.matmul(L1, W2), b2)

In [11]:
model = tf.nn.softmax(model)

In [12]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)

In [18]:
train_op = optimizer.minimize(cost)

In [19]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(10000):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if (step+1) % 100 == 0:
        print(step+1, sess.run(cost, feed_dict={X: x_data, Y:y_data}))

100 1.0984578
200 1.0983061
300 1.098172
400 1.098035
500 1.0979223
600 1.097809
700 1.0977042
800 1.0976025
900 1.0975131
1000 1.0974286


In [20]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)

print('에측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

에측값: [2 2 2 ... 2 2 2]
실제값: [2 2 1 ... 0 0 1]
정확도: 36.61
